# This notebook takes the combined data and references produced from get_references and performs post-processing of recorded data including:
1. Cleans numerical value strings. 
2. Normalizes recorded compositions.
3. Generates classifiers used for visualizations.
4. Calculates density and Young modulus.

In [1]:
import os
import re
import pandas as pd
import numpy as np
import pymatgen as mg

def categorize_phases(row):
    '''Creates BCC/FCC/other classifier used for visualizations'''
    
    if row['PROPERTY: Type of phases'] == 'BCC':
        val = 'BCC'
    elif row['PROPERTY: Type of phases'] == 'FCC':
        val = 'FCC'
    else:
        val = 'other'
    return val


def remove_uncertainty(cell):
    '''Sanitizes values given as range '''
    
    val = str(cell)
    
    if '$\pm$' in str(cell):
        val = cell.split('$\pm$')[0]
    if '>' in val:
        val = val.replace('>', '')
    if '<' in val:
        val = val.replace('<', '')
    if 'nan' in val:
        val = ''
        
    return val


def average_range(cell):
    '''Returns average if given range of values.'''
    
    val = str(cell).replace(" ", "")
    
    if re.findall(r"\b\d+-\d+\b", val):
        data = [float(x) for x in val.split('-')]
        val = np.average(data)
        
    return val


def neg_elongation(row):
    '''Assigns a negative sign for compression data. Used for visualization purposes.'''
    
    if row['PROPERTY: Type of test'] == "C" and row['PROPERTY: Elongation (%)'] != "":
        val = -float(row['PROPERTY: Elongation (%)'])
    else:
        val = row['PROPERTY: Elongation (%)']
    return val


def normalize_and_alphabetize_formula(formula):
    '''Normalizes composition labels. Used to enable matching / groupby on compositions.'''
    
    if formula:
        try:
            comp = mg.Composition(formula)
            weights = [comp.get_atomic_fraction(ele) for ele in comp.elements]
            normalized_weights = [round(w/max(weights), 3) for w in weights]
            normalized_comp = "".join([str(x)+str(y) for x,y in zip(comp.elements, normalized_weights)])
            
            return mg.Composition(normalized_comp).alphabetical_formula
        except:
            print("INVALID: ", formula)
            return None
    else:
        return None
    
def standardize_synthesis_method(synth_method):
    '''Standardizes synthesis method strings.'''
    
    return synth_method.replace(" ", "")


def calculate_density(formula):
    '''Calculates densisty based on Rule of Mixtures (ROM).'''
    
    comp = mg.Composition(formula)
    
    weights = [comp.get_atomic_fraction(e)for e in comp.elements]
    vols = np.array([e.molar_volume for e in comp.elements])
    atomic_masses = np.array([e.atomic_mass for e in comp.elements])
    
    val = np.sum(weights*atomic_masses) / np.sum(weights*vols)

    return round(val, 1)


def calculate_youngs_modulus(formula):
    '''Calculates Young Modulus based on Rule of Mixtures (ROM).'''
    
    comp = mg.Composition(formula)
    
    weights = np.array([comp.get_atomic_fraction(e)for e in comp.elements])
    vols = np.array([e.molar_volume for e in comp.elements])
    ym_vals = np.array([e.youngs_modulus for e in comp.elements])
    
    if None in ym_vals:
        return ''
    
    val = np.sum(weights*vols*ym_vals) / np.sum(weights*vols)
    
    return int(round(val, 0))


def classify_processing_method(process_method):
    '''Creates a processing method classifier based on recorded synthesis method.'''
    
    if process_method == 'AC' or process_method == 'DC':
        return 'CAST'
    
    if 'SPD' in process_method or 'AM' in process_method or 'HIP' in process_method:
        return 'OTHER'
    
    if 'GA' in process_method or 'MA' in process_method or 'SPS' in process_method or 'VHP' in process_method or process_method == 'S':
        return 'POWDER'
    
    if 'CR' in process_method or 'HR' in process_method or 'HF' in process_method:
        return 'WROUGHT'
    
    if process_method == 'A' or process_method == 'H' or '+A' in process_method or '+H' in process_method or 'Aged' in process_method:
        return 'ANNEAL'


def classify_microstructure(phases):
    '''Creates a microstructure classifier based on recorded phases.'''
    
    phases = phases.replace(' ', '')
    
    valid_phases = ['FCC', 'BCC', 'HCP', 'L12', 'B2', 'Laves', 'Laves (C14)', 'Laves (C15)']
    
    if phases == '':
        return ''
    
    phase_list = phases.split('+')
    
    if len(phase_list) == 1 and phase_list[0] in valid_phases:
        return phases
    elif len(phase_list) == 1 and phase_list[0] not in valid_phases:
        return 'Other'
    
    if len(phase_list) > 1 and len(set(phase_list)) == 1 and phase_list[0] in valid_phases:
        return phases
    elif len(phase_list) > 1 and len(set(phase_list)) == 1 and phase_list[0] not in valid_phases:
        return 'Other'
    
    if len(phase_list) > 1 and len(set(phase_list)) > 1 and phase_list[0] in valid_phases:

        
        # if all phases are a subset of valid phases, return all phases
        if set(phase_list).issubset(set(valid_phases)):
            return phases
        else:
            sub_valid = []
            for phase in phase_list:
                if phase in valid_phases:
                    sub_valid.append(phase)
            
            # return valid phases + invalid labeled as "Sec."
            if len(sub_valid) > 0:
                return '+'.join(sub_valid)+"+Sec."
           
            # no valid phases
            else:
                return 'Other'
    else:
        return 'Other'

In [2]:
# import combined data, add refs
df = pd.read_csv('combined_data.csv')
df_refs = pd.read_csv('references/references.csv')
df = df.merge(df_refs, on='IDENTIFIER: Reference ID', how='left')
df['REFERENCE: doi'] = df['REFERENCE: doi_y']
df

,Unnamed: 0,REFERENCE: tag,REFERENCE: doi_x,FORMULA,PROPERTY: Type of phases,PROPERTY: Single/Multiphase,PROPERTY: synthesis method,PROPERTY: grain size ($\mu$m),PROPERTY: ROM Density (g/cm$^3$),PROPERTY: HV,...,Internal Reference #,Original DOI,PROPERTY: Exp. Density (g/cm$^3$),Unnamed: 22,IDENTIFIER: Reference ID,REFERENCE: title,REFERENCE: year,REFERENCE: doi_y,REFERENCE: url,REFERENCE: doi
0,0,4,10.1016/j.jmmm.2014.07.023,Al0.25CoFeNi,FCC,S,AC,NaN,7.9,138.0,...,NaN,NaN,NaN,NaN,27,Effects of Al and Si addition on the structure...,2014,10.1016/j.jmmm.2014.07.023,http://dx.doi.org/10.1016/j.jmmm.2014.07.023,10.1016/j.jmmm.2014.07.023
1,1,4,10.1016/j.jmmm.2014.07.023,Al0.5CoFeNi,FCC+BCC,M,AC,NaN,7.4,212.0,...,NaN,NaN,NaN,NaN,27,Effects of Al and Si addition on the structure...,2014,10.1016/j.jmmm.2014.07.023,http://dx.doi.org/10.1016/j.jmmm.2014.07.023,10.1016/j.jmmm.2014.07.023
2,2,4,10.1016/j.jmmm.2014.07.023,Al0.75CoFeNi,FCC+BCC,M,AC,NaN,7.0,385.0,...,NaN,NaN,NaN,NaN,27,Effects of Al and Si addition on the structure...,2014,10.1016/j.jmmm.2014.07.023,http://dx.doi.org/10.1016/j.jmmm.2014.07.023,10.1016/j.jmmm.2014.07.023
3,3,4,10.1016/j.jmmm.2014.07.023,AlCoFeNi,BCC,S,AC,NaN,6.6,456.0,...,NaN,NaN,NaN,NaN,27,Effects of Al and Si addition on the structure...,2014,10.1016/j.jmmm.2014.07.023,http://dx.doi.org/10.1016/j.jmmm.2014.07.023,10.1016/j.jmmm.2014.07.023
4,4,4,10.1016/j.jmmm.2014.07.023,CoFeNi,FCC,S,AC,NaN,8.5,125.0,...,NaN,NaN,NaN,NaN,27,Effects of Al and Si addition on the structure...,2014,10.1016/j.jmmm.2014.07.023,http://dx.doi.org/10.1016/j.jmmm.2014.07.023,10.1016/j.jmmm.2014.07.023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545,931,new_ref_4,10.1016/j.matdes.2018.06.003,HfNbTaTi,BCC,S,AC,NaN,NaN,270 $\pm$ 5,...,NaN,NaN,NaN,NaN,266,"Phase equilibria, mechanical properties and de...",2018,10.1016/j.matdes.2018.06.003,http://dx.doi.org/10.1016/j.matdes.2018.06.003,10.1016/j.matdes.2018.06.003
1546,932,new_ref_4,10.1016/j.matdes.2018.06.003,MoNbTaTiW,BCC,S,A,NaN,NaN,482 $\pm$ 10,...,NaN,NaN,NaN,NaN,266,"Phase equilibria, mechanical properties and de...",2018,10.1016/j.matdes.2018.06.003,http://dx.doi.org/10.1016/j.matdes.2018.06.003,10.1016/j.matdes.2018.06.003
1547,933,new_ref_4,10.1016/j.matdes.2018.06.003,MoNbTaTiW,BCC,S,AC,NaN,NaN,446 $\pm$ 32,...,NaN,NaN,NaN,NaN,266,"Phase equilibria, mechanical properties and de...",2018,10.1016/j.matdes.2018.06.003,http://dx.doi.org/10.1016/j.matdes.2018.06.003,10.1016/j.matdes.2018.06.003
1548,934,new_ref_4,10.1016/j.matdes.2018.06.003,MoNbTaTi,BCC,S,A,NaN,NaN,407 $\pm$ 17,...,NaN,NaN,NaN,NaN,266,"Phase equilibria, mechanical properties and de...",2018,10.1016/j.matdes.2018.06.003,http://dx.doi.org/10.1016/j.matdes.2018.06.003,10.1016/j.matdes.2018.06.003


In [3]:
# remove uncertainties / clean numerical values
df = df.applymap(remove_uncertainty)
df['PROPERTY: grain size ($\mu$m)'] = df['PROPERTY: grain size ($\mu$m)'].apply(average_range)
df['PROPERTY: Elongation (%)'] = df['PROPERTY: Elongation (%)'].apply(average_range)

# process columns to ensure data types are accurate
numeric_props = ['PROPERTY: grain size ($\mu$m)', 'PROPERTY: ROM Density (g/cm$^3$)', 'PROPERTY: HV', 'PROPERTY: Test temperature ($^\circ$C)',
                'PROPERTY: YS (MPa)', 'PROPERTY: UTS (MPa)', 'PROPERTY: Elongation (%)', 'PROPERTY: Exp. Young modulus (GPa)']

df[numeric_props] = df[numeric_props].apply(pd.to_numeric)

df

,Unnamed: 0,REFERENCE: tag,REFERENCE: doi_x,FORMULA,PROPERTY: Type of phases,PROPERTY: Single/Multiphase,PROPERTY: synthesis method,PROPERTY: grain size ($\mu$m),PROPERTY: ROM Density (g/cm$^3$),PROPERTY: HV,...,Internal Reference #,Original DOI,PROPERTY: Exp. Density (g/cm$^3$),Unnamed: 22,IDENTIFIER: Reference ID,REFERENCE: title,REFERENCE: year,REFERENCE: doi_y,REFERENCE: url,REFERENCE: doi
0,0,4,10.1016/j.jmmm.2014.07.023,Al0.25CoFeNi,FCC,S,AC,NaN,7.9,138.0,...,,,,,27,Effects of Al and Si addition on the structure...,2014,10.1016/j.jmmm.2014.07.023,http://dx.doi.org/10.1016/j.jmmm.2014.07.023,10.1016/j.jmmm.2014.07.023
1,1,4,10.1016/j.jmmm.2014.07.023,Al0.5CoFeNi,FCC+BCC,M,AC,NaN,7.4,212.0,...,,,,,27,Effects of Al and Si addition on the structure...,2014,10.1016/j.jmmm.2014.07.023,http://dx.doi.org/10.1016/j.jmmm.2014.07.023,10.1016/j.jmmm.2014.07.023
2,2,4,10.1016/j.jmmm.2014.07.023,Al0.75CoFeNi,FCC+BCC,M,AC,NaN,7.0,385.0,...,,,,,27,Effects of Al and Si addition on the structure...,2014,10.1016/j.jmmm.2014.07.023,http://dx.doi.org/10.1016/j.jmmm.2014.07.023,10.1016/j.jmmm.2014.07.023
3,3,4,10.1016/j.jmmm.2014.07.023,AlCoFeNi,BCC,S,AC,NaN,6.6,456.0,...,,,,,27,Effects of Al and Si addition on the structure...,2014,10.1016/j.jmmm.2014.07.023,http://dx.doi.org/10.1016/j.jmmm.2014.07.023,10.1016/j.jmmm.2014.07.023
4,4,4,10.1016/j.jmmm.2014.07.023,CoFeNi,FCC,S,AC,NaN,8.5,125.0,...,,,,,27,Effects of Al and Si addition on the structure...,2014,10.1016/j.jmmm.2014.07.023,http://dx.doi.org/10.1016/j.jmmm.2014.07.023,10.1016/j.jmmm.2014.07.023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545,931,new_ref_4,10.1016/j.matdes.2018.06.003,HfNbTaTi,BCC,S,AC,NaN,NaN,270.0,...,,,,,266,"Phase equilibria, mechanical properties and de...",2018,10.1016/j.matdes.2018.06.003,http://dx.doi.org/10.1016/j.matdes.2018.06.003,10.1016/j.matdes.2018.06.003
1546,932,new_ref_4,10.1016/j.matdes.2018.06.003,MoNbTaTiW,BCC,S,A,NaN,NaN,482.0,...,,,,,266,"Phase equilibria, mechanical properties and de...",2018,10.1016/j.matdes.2018.06.003,http://dx.doi.org/10.1016/j.matdes.2018.06.003,10.1016/j.matdes.2018.06.003
1547,933,new_ref_4,10.1016/j.matdes.2018.06.003,MoNbTaTiW,BCC,S,AC,NaN,NaN,446.0,...,,,,,266,"Phase equilibria, mechanical properties and de...",2018,10.1016/j.matdes.2018.06.003,http://dx.doi.org/10.1016/j.matdes.2018.06.003,10.1016/j.matdes.2018.06.003
1548,934,new_ref_4,10.1016/j.matdes.2018.06.003,MoNbTaTi,BCC,S,A,NaN,NaN,407.0,...,,,,,266,"Phase equilibria, mechanical properties and de...",2018,10.1016/j.matdes.2018.06.003,http://dx.doi.org/10.1016/j.matdes.2018.06.003,10.1016/j.matdes.2018.06.003


In [4]:
# ensure formulas are valid
df['FORMULA'] = df['FORMULA'].apply(normalize_and_alphabetize_formula)

# add classifiers
df['PROPERTY: BCC/FCC/other'] = df.apply(categorize_phases, axis=1)
df['PROPERTY: Processing method'] = df['PROPERTY: synthesis method'].apply(standardize_synthesis_method)
df['PROPERTY: Processing method'] = df['PROPERTY: Processing method'].apply(classify_processing_method)
df['PROPERTY: Microstructure'] = df['PROPERTY: Type of phases'].apply(classify_microstructure)

# calculate props
df['PROPERTY: Calculated Density (g/cm$^3$)'] = df['FORMULA'].apply(calculate_density)
df['PROPERTY: Calculated Young modulus (GPa)'] = df.apply(lambda x: calculate_youngs_modulus(x['FORMULA']) if x['PROPERTY: BCC/FCC/other'] != 'other' else '', axis=1)

df.to_csv(os.path.abspath('stats_and_tables/MPEA_dataset_for_stats.csv'), index=False)
df

,Unnamed: 0,REFERENCE: tag,REFERENCE: doi_x,FORMULA,PROPERTY: Type of phases,PROPERTY: Single/Multiphase,PROPERTY: synthesis method,PROPERTY: grain size ($\mu$m),PROPERTY: ROM Density (g/cm$^3$),PROPERTY: HV,...,REFERENCE: title,REFERENCE: year,REFERENCE: doi_y,REFERENCE: url,REFERENCE: doi,PROPERTY: BCC/FCC/other,PROPERTY: Processing method,PROPERTY: Microstructure,PROPERTY: Calculated Density (g/cm$^3$),PROPERTY: Calculated Young modulus (GPa)
0,0,4,10.1016/j.jmmm.2014.07.023,Al0.25 Co1 Fe1 Ni1,FCC,S,AC,NaN,7.9,138.0,...,Effects of Al and Si addition on the structure...,2014,10.1016/j.jmmm.2014.07.023,http://dx.doi.org/10.1016/j.jmmm.2014.07.023,10.1016/j.jmmm.2014.07.023,FCC,CAST,FCC,7.9,192
1,1,4,10.1016/j.jmmm.2014.07.023,Al0.5 Co1 Fe1 Ni1,FCC+BCC,M,AC,NaN,7.4,212.0,...,Effects of Al and Si addition on the structure...,2014,10.1016/j.jmmm.2014.07.023,http://dx.doi.org/10.1016/j.jmmm.2014.07.023,10.1016/j.jmmm.2014.07.023,other,CAST,FCC+BCC,7.4,
2,2,4,10.1016/j.jmmm.2014.07.023,Al0.75 Co1 Fe1 Ni1,FCC+BCC,M,AC,NaN,7.0,385.0,...,Effects of Al and Si addition on the structure...,2014,10.1016/j.jmmm.2014.07.023,http://dx.doi.org/10.1016/j.jmmm.2014.07.023,10.1016/j.jmmm.2014.07.023,other,CAST,FCC+BCC,7.0,
3,3,4,10.1016/j.jmmm.2014.07.023,Al1 Co1 Fe1 Ni1,BCC,S,AC,NaN,6.6,456.0,...,Effects of Al and Si addition on the structure...,2014,10.1016/j.jmmm.2014.07.023,http://dx.doi.org/10.1016/j.jmmm.2014.07.023,10.1016/j.jmmm.2014.07.023,BCC,CAST,BCC,6.6,162
4,4,4,10.1016/j.jmmm.2014.07.023,Co1 Fe1 Ni1,FCC,S,AC,NaN,8.5,125.0,...,Effects of Al and Si addition on the structure...,2014,10.1016/j.jmmm.2014.07.023,http://dx.doi.org/10.1016/j.jmmm.2014.07.023,10.1016/j.jmmm.2014.07.023,FCC,CAST,FCC,8.5,207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545,931,new_ref_4,10.1016/j.matdes.2018.06.003,Hf1 Nb1 Ta1 Ti1,BCC,S,AC,NaN,NaN,270.0,...,"Phase equilibria, mechanical properties and de...",2018,10.1016/j.matdes.2018.06.003,http://dx.doi.org/10.1016/j.matdes.2018.06.003,10.1016/j.matdes.2018.06.003,BCC,CAST,BCC,10.9,119
1546,932,new_ref_4,10.1016/j.matdes.2018.06.003,Mo1 Nb1 Ta1 Ti1 W1,BCC,S,A,NaN,NaN,482.0,...,"Phase equilibria, mechanical properties and de...",2018,10.1016/j.matdes.2018.06.003,http://dx.doi.org/10.1016/j.matdes.2018.06.003,10.1016/j.matdes.2018.06.003,BCC,ANNEAL,BCC,11.8,222
1547,933,new_ref_4,10.1016/j.matdes.2018.06.003,Mo1 Nb1 Ta1 Ti1 W1,BCC,S,AC,NaN,NaN,446.0,...,"Phase equilibria, mechanical properties and de...",2018,10.1016/j.matdes.2018.06.003,http://dx.doi.org/10.1016/j.matdes.2018.06.003,10.1016/j.matdes.2018.06.003,BCC,CAST,BCC,11.8,222
1548,934,new_ref_4,10.1016/j.matdes.2018.06.003,Mo1 Nb1 Ta1 Ti1,BCC,S,A,NaN,NaN,407.0,...,"Phase equilibria, mechanical properties and de...",2018,10.1016/j.matdes.2018.06.003,http://dx.doi.org/10.1016/j.matdes.2018.06.003,10.1016/j.matdes.2018.06.003,BCC,ANNEAL,BCC,10.0,179


In [5]:
# define columns of interest, output dataset file
cols = ['IDENTIFIER: Reference ID', 'FORMULA', 'PROPERTY: Microstructure', 'PROPERTY: Processing method', 'PROPERTY: BCC/FCC/other', 'PROPERTY: grain size ($\\mu$m)',
        'PROPERTY: Exp. Density (g/cm$^3$)', 'PROPERTY: Calculated Density (g/cm$^3$)', 'PROPERTY: HV', 'PROPERTY: Type of test', 'PROPERTY: Test temperature ($^\\circ$C)', 'PROPERTY: YS (MPa)', 'PROPERTY: UTS (MPa)', 
        'PROPERTY: Elongation (%)', 'PROPERTY: Elongation plastic (%)', 'PROPERTY: Exp. Young modulus (GPa)', 'PROPERTY: Calculated Young modulus (GPa)', 'PROPERTY: O content (wppm)', 'PROPERTY: N content (wppm)', 
        'PROPERTY: C content (wppm)', 'REFERENCE: doi', 'REFERENCE: year', 'REFERENCE: title']

df_output = df[cols]
df_output.to_csv(os.path.abspath('MPEA_dataset.csv'), index=False)
df_output

,IDENTIFIER: Reference ID,FORMULA,PROPERTY: Microstructure,PROPERTY: Processing method,PROPERTY: BCC/FCC/other,PROPERTY: grain size ($\mu$m),PROPERTY: Exp. Density (g/cm$^3$),PROPERTY: Calculated Density (g/cm$^3$),PROPERTY: HV,PROPERTY: Type of test,...,PROPERTY: Elongation (%),PROPERTY: Elongation plastic (%),PROPERTY: Exp. Young modulus (GPa),PROPERTY: Calculated Young modulus (GPa),PROPERTY: O content (wppm),PROPERTY: N content (wppm),PROPERTY: C content (wppm),REFERENCE: doi,REFERENCE: year,REFERENCE: title
0,27,Al0.25 Co1 Fe1 Ni1,FCC,CAST,FCC,NaN,,7.9,138.0,C,...,NaN,,NaN,192,,,,10.1016/j.jmmm.2014.07.023,2014,Effects of Al and Si addition on the structure...
1,27,Al0.5 Co1 Fe1 Ni1,FCC+BCC,CAST,other,NaN,,7.4,212.0,C,...,NaN,,NaN,,,,,10.1016/j.jmmm.2014.07.023,2014,Effects of Al and Si addition on the structure...
2,27,Al0.75 Co1 Fe1 Ni1,FCC+BCC,CAST,other,NaN,,7.0,385.0,C,...,NaN,,NaN,,,,,10.1016/j.jmmm.2014.07.023,2014,Effects of Al and Si addition on the structure...
3,27,Al1 Co1 Fe1 Ni1,BCC,CAST,BCC,NaN,,6.6,456.0,C,...,NaN,,NaN,162,,,,10.1016/j.jmmm.2014.07.023,2014,Effects of Al and Si addition on the structure...
4,27,Co1 Fe1 Ni1,FCC,CAST,FCC,NaN,,8.5,125.0,C,...,NaN,,NaN,207,,,,10.1016/j.jmmm.2014.07.023,2014,Effects of Al and Si addition on the structure...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545,266,Hf1 Nb1 Ta1 Ti1,BCC,CAST,BCC,NaN,,10.9,270.0,,...,NaN,,NaN,119,,,,10.1016/j.matdes.2018.06.003,2018,"Phase equilibria, mechanical properties and de..."
1546,266,Mo1 Nb1 Ta1 Ti1 W1,BCC,ANNEAL,BCC,NaN,,11.8,482.0,,...,NaN,,NaN,222,,,,10.1016/j.matdes.2018.06.003,2018,"Phase equilibria, mechanical properties and de..."
1547,266,Mo1 Nb1 Ta1 Ti1 W1,BCC,CAST,BCC,NaN,,11.8,446.0,,...,NaN,,NaN,222,,,,10.1016/j.matdes.2018.06.003,2018,"Phase equilibria, mechanical properties and de..."
1548,266,Mo1 Nb1 Ta1 Ti1,BCC,ANNEAL,BCC,NaN,,10.0,407.0,,...,NaN,,NaN,179,,,,10.1016/j.matdes.2018.06.003,2018,"Phase equilibria, mechanical properties and de..."
